In [28]:
#Muhamad Wino Jati Pinulung

import numpy as np
import random
from typing import Tuple
import csv

In [29]:
# Definisi lingkungan sederhana untuk simulasi trading saham
class LingkunganSahamSederhana:
    def __init__(self):
        # State: [kepemilikan_saham, tren_harga]
        # kepemilikan_saham: 0 (tidak punya) atau 1 (punya)
        # tren_harga: 0 (turun) atau 1 (naik)
        self.jumlah_state = 4  # 2 kepemilikan × 2 tren
        self.jumlah_aksi = 2  # 0: tunggu, 1: beli/jual
        
    def reset(self) -> int:
        # Inisialisasi ulang lingkungan
        self.kepemilikan = 0  # tidak punya saham
        self.tren_harga = random.randint(0, 1)  # tren harga random
        return self._ambil_state()
    
    def _ambil_state(self) -> int:
        # Menghitung state berdasarkan kepemilikan saham dan tren harga
        return self.kepemilikan * 2 + self.tren_harga
    
    def langkah(self, aksi: int) -> Tuple[int, float, bool]:
        # Fungsi untuk menjalankan satu langkah dalam lingkungan
        reward = 0
        
        # Proses aksi
        if aksi == 1:  # beli/jual
            if self.kepemilikan == 0:  # beli
                reward = 1 if self.tren_harga == 0 else -1  # untung jika beli saat turun
                self.kepemilikan = 1
            else:  # jual
                reward = 1 if self.tren_harga == 1 else -1  # untung jika jual saat naik
                self.kepemilikan = 0
        
        # Update tren harga (50% kemungkinan berubah)
        if random.random() < 0.5:
            self.tren_harga = 1 - self.tren_harga
        
        # 10% kemungkinan episode berakhir
        selesai = random.random() < 0.1
        
        return self._ambil_state(), reward, selesai
                

In [30]:
# Algoritma Q-Learning sederhana
class QLearningSederhana:
    def __init__(self, jumlah_state: int, jumlah_aksi: int, 
                 kecepatan_belajar: float = 0.1, 
                 faktor_diskon: float = 0.95, 
                 epsilon: float = 0.1):
        # Inisialisasi tabel Q dengan nilai nol
        self.tabel_q = np.zeros((jumlah_state, jumlah_aksi))
        self.lr = kecepatan_belajar  # learning rate
        self.gamma = faktor_diskon  # discount factor
        self.epsilon = epsilon  # eksplorasi vs eksploitasi
    
    def ambil_aksi(self, state: int, training: bool = True) -> int:
        # Mengambil aksi berdasarkan kebijakan eksplorasi/eksploitasi
        if training and random.random() < self.epsilon:
            return random.randint(0, 1)  # eksplorasi
        return np.argmax(self.tabel_q[state])  # eksploitasi
    
    def update(self, state: int, aksi: int, reward: float, state_berikutnya: int):
        # Update tabel Q menggunakan formula Q-Learning
        nilai_lama = self.tabel_q[state, aksi]
        nilai_maks_berikutnya = np.max(self.tabel_q[state_berikutnya])
        self.tabel_q[state, aksi] = (1 - self.lr) * nilai_lama + \
                                    self.lr * (reward + self.gamma * nilai_maks_berikutnya)



In [31]:
# Fungsi untuk melatih agen menggunakan lingkungan
def latih_agen(lingkungan: LingkunganSahamSederhana, agen: QLearningSederhana, 
               jumlah_episode: int = 1000) -> list:
    sejarah_reward = []
    
    for episode in range(jumlah_episode):
        state = lingkungan.reset()
        reward_episode = 0
        selesai = False
        
        while not selesai:
            aksi = agen.ambil_aksi(state)
            state_berikutnya, reward, selesai = lingkungan.langkah(aksi)
            agen.update(state, aksi, reward, state_berikutnya)
            state = state_berikutnya
            reward_episode += reward
            
        sejarah_reward.append(reward_episode)
        
        # Print progress setiap 200 episode
        if (episode + 1) % 200 == 0:
            rata2_reward = np.mean(sejarah_reward[-200:])
            print(f"Episode {episode + 1}, Rata-rata reward: {rata2_reward:.2f}")
    
    return sejarah_reward


In [32]:
# Fungsi untuk menguji performa agen
def uji_agen(lingkungan: LingkunganSahamSederhana, agen: QLearningSederhana, 
             jumlah_episode: int = 5) -> None:
    
    def deskripsi_state(state: int) -> str:
        # Mengembalikan deskripsi state untuk kejelasan
        kepemilikan = "Punya saham" if state // 2 else "Tidak punya saham"
        tren = "Naik" if state % 2 else "Turun"
        return f"{kepemilikan}, Harga: {tren}"
    
    def deskripsi_aksi(aksi: int) -> str:
        # Mengembalikan deskripsi aksi
        return "Beli/Jual" if aksi == 1 else "Tunggu"
    
    print("\nMulai testing...")
    for episode in range(jumlah_episode):
        state = lingkungan.reset()
        reward_episode = 0
        langkah = 0
        selesai = False
        
        print(f"\nEpisode {episode + 1}")
        print("-" * 40)
        
        while not selesai and langkah < 10:  # maksimum 10 langkah per episode
            aksi = agen.ambil_aksi(state, training=False)
            
            print(f"\nLangkah {langkah + 1}")
            print(f"State: {deskripsi_state(state)}")
            print(f"Aksi: {deskripsi_aksi(aksi)}")
            
            state_berikutnya, reward, selesai = lingkungan.langkah(aksi)
            print(f"Reward: {reward}")
            
            state = state_berikutnya
            reward_episode += reward
            langkah += 1
        
        print(f"\nTotal reward episode: {reward_episode}")
        print(f"Jumlah langkah: {langkah}")


In [33]:
if __name__ == "__main__":
    # Inisialisasi lingkungan dan agen
    lingkungan = LingkunganSahamSederhana()
    agen = QLearningSederhana(lingkungan.jumlah_state, lingkungan.jumlah_aksi)
    
    # Training agen
    print("Mulai training...")
    rewards = latih_agen(lingkungan, agen, jumlah_episode=1000)
    print("\nTraining selesai!")
    
    # Testing agen
    uji_agen(lingkungan, agen, jumlah_episode=5)
    
    # Menampilkan Q-table akhir
    print("\nQ-table akhir:")
    print(agen.tabel_q)

Mulai training...
Episode 200, Rata-rata reward: 4.08
Episode 400, Rata-rata reward: 4.57
Episode 600, Rata-rata reward: 4.42
Episode 800, Rata-rata reward: 4.46
Episode 1000, Rata-rata reward: 5.10

Training selesai!

Mulai testing...

Episode 1
----------------------------------------

Langkah 1
State: Tidak punya saham, Harga: Turun
Aksi: Beli/Jual
Reward: 1

Langkah 2
State: Punya saham, Harga: Naik
Aksi: Beli/Jual
Reward: 1

Langkah 3
State: Tidak punya saham, Harga: Turun
Aksi: Beli/Jual
Reward: 1

Langkah 4
State: Punya saham, Harga: Turun
Aksi: Tunggu
Reward: 0

Langkah 5
State: Punya saham, Harga: Naik
Aksi: Beli/Jual
Reward: 1

Langkah 6
State: Tidak punya saham, Harga: Naik
Aksi: Tunggu
Reward: 0

Langkah 7
State: Tidak punya saham, Harga: Naik
Aksi: Tunggu
Reward: 0

Langkah 8
State: Tidak punya saham, Harga: Naik
Aksi: Tunggu
Reward: 0

Langkah 9
State: Tidak punya saham, Harga: Turun
Aksi: Beli/Jual
Reward: 1

Langkah 10
State: Punya saham, Harga: Turun
Aksi: Tunggu
Rewar

In [41]:
class SimpleStockEnv:
    def __init__(self):
        self.n_states = 4  # 2 kepemilikan × 2 tren
        self.n_actions = 2  # 0: tunggu, 1: beli/jual
        
    def reset(self) -> int:
        self.owned = 0  # tidak punya saham
        self.price_trend = random.randint(0, 1)  # tren harga random
        return self._get_state()
    
    def _get_state(self) -> int:
        return self.owned * 2 + self.price_trend
    
    def step(self, action: int) -> Tuple[int, float, bool]:
        reward = 0
        
        # Proses aksi
        if action == 1:  # beli/jual
            if self.owned == 0:  # beli
                reward = 1 if self.price_trend == 0 else -1  # untung jika beli saat turun
                self.owned = 1
            else:  # jual
                reward = 1 if self.price_trend == 1 else -1  # untung jika jual saat naik
                self.owned = 0
        
        # Update tren harga (50% kemungkinan berubah)
        if random.random() < 0.5:
            self.price_trend = 1 - self.price_trend
        
        # 10% kemungkinan episode berakhir
        done = random.random() < 0.1
        
        return self._get_state(), reward, done


In [42]:
# Inisialisasi environment
env = SimpleStockEnv()

In [43]:
# Buat dataset
dataset = []

In [46]:
# Tentukan jumlah episode
n_episodes = 1000

In [49]:
# Generate data untuk 1000 episode
for episode in range(n_episodes):
    state = env.reset()
    done = False
    while not done:
        action = random.randint(0, 1)  # Pilih aksi secara acak untuk simulasi
        next_state, reward, done = env.step(action)
        
        # Simpan data
        dataset.append([state, action, reward, next_state, done])

In [51]:
# Simpan dataset dalam format CSV
with open('q_learning_dataset.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["State", "Action", "Reward", "Next State", "Done"])  # Menulis header
    writer.writerows(dataset)  # Menulis data

print(f"Dataset berhasil disimpan dalam 'q_learning_dataset'.")

Dataset berhasil disimpan dalam 'q_learning_dataset'.
